In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore")

url_hourly = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/transformers_dataset.csv"
df_hourly = pd.read_csv(url_hourly,  sep=';', encoding='latin-1')

url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily,  sep=';', encoding='latin-1')


In [ ]:
# ========================================================================
# SISTEMA DE ANÁLISE E RELATÓRIO DE TENDÊNCIAS - VERSÃO CORRIGIDA
# ========================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.size'] = 12
plt.style.use('seaborn-v0_8')

class AnalisadorTendencias:
    def __init__(self, data):
        self.data = data.copy()
        self.data['datahora'] = pd.to_datetime(self.data['datahora'])
        self._preprocessar_dados()
    
    def _preprocessar_dados(self):
        """Preprocessamento dos dados para análise"""
        # Extrair componentes de tempo
        self.data['hora'] = self.data['datahora'].dt.hour
        self.data['dia_semana'] = self.data['datahora'].dt.day_name()
        self.data['mes'] = self.data['datahora'].dt.month
        self.data['trimestre'] = self.data['datahora'].dt.quarter
        self.data['ano'] = self.data['datahora'].dt.year
        
        # Classificar horários do dia
        condicoes = [
            (self.data['hora'] >= 6) & (self.data['hora'] < 12),   # Manhã
            (self.data['hora'] >= 12) & (self.data['hora'] < 18),  # Tarde
            (self.data['hora'] >= 18) & (self.data['hora'] < 24),  # Noite
            (self.data['hora'] >= 0) & (self.data['hora'] < 6)     # Madrugada
        ]
        categorias = ['Manhã (6h-12h)', 'Tarde (12h-18h)', 'Noite (18h-24h)', 'Madrugada (0h-6h)']
        self.data['periodo_dia'] = np.select(condicoes, categorias, default='Madrugada (0h-6h)')
        
        # Classificar dias da semana
        dias_trabalho = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        self.data['tipo_dia'] = self.data['dia_semana'].apply(
            lambda x: 'Dia de Trabalho' if x in dias_trabalho else 'Fim de Semana'
        )
    
    def analisar_tendencias_trafo(self, trafo_id):
        """Análise completa de tendências para um transformador"""
        print(f"\n{'='*80}")
        print(f"🔍 ANÁLISE DETALHADA - TRAFO {trafo_id}")
        print(f"{'='*80}")
        
        df_trafo = self.data[self.data['id'] == trafo_id].copy()
        
        if len(df_trafo) == 0:
            print(f"❌ Nenhum dado encontrado para o trafo {trafo_id}")
            return None
        
        try:
            resultados = {
                'trafo': trafo_id,
                'analise_horaria': self._analise_horaria(df_trafo),
                'analise_diaria': self._analise_diaria(df_trafo),
                'analise_mensal': self._analise_mensal(df_trafo),
                'padroes_sazonais': self._analise_sazonal(df_trafo),
                'alertas': self._gerar_alertas(df_trafo)
            }
            
            self._gerar_relatorio_visual(resultados, df_trafo)
            self._exportar_relatorio_textual(resultados)
            
            return resultados
            
        except Exception as e:
            print(f"❌ Erro na análise do trafo {trafo_id}: {str(e)}")
            import traceback
            traceback.print_exc()
            return None
    
    def _analise_horaria(self, df):
        """Análise de padrões horários - VERSÃO CORRIGIDA"""
        analise_hora = df.groupby('hora')['S'].agg(['mean', 'std', 'max', 'min']).round(2)
        
        # Identificar picos e vales - CORREÇÃO AQUI
        if not analise_hora.empty:
            hora_pico = analise_hora['mean'].idxmax()
            hora_vale = analise_hora['mean'].idxmin()
            carga_pico = float(analise_hora.loc[hora_pico, 'mean'])
            carga_vale = float(analise_hora.loc[hora_vale, 'mean'])
        else:
            hora_pico = hora_vale = 0
            carga_pico = carga_vale = 0.0
        
        # Análise por período do dia
        analise_periodo = df.groupby('periodo_dia')['S'].agg(['mean', 'std']).round(2)
        
        return {
            'hora_pico': hora_pico,
            'carga_pico': carga_pico,
            'hora_vale': hora_vale,
            'carga_vale': carga_vale,
            'amplitude_diaria': round(carga_pico - carga_vale, 2),
            'analise_periodo': analise_periodo,
            'tendencia_horaria': self._calcular_tendencia_horaria(df)  # CORREÇÃO AQUI
        }
    
    def _analise_diaria(self, df):
        """Análise de padrões por dia da semana"""
        analise_dia = df.groupby('dia_semana')['S'].agg(['mean', 'std']).round(2)
        analise_tipo_dia = df.groupby('tipo_dia')['S'].agg(['mean', 'std']).round(2)
        
        if not analise_dia.empty:
            dia_maior_carga = analise_dia['mean'].idxmax()
            dia_menor_carga = analise_dia['mean'].idxmin()
            
            if 'Dia de Trabalho' in analise_tipo_dia.index and 'Fim de Semana' in analise_tipo_dia.index:
                diferenca = float(analise_tipo_dia.loc['Dia de Trabalho', 'mean'] - analise_tipo_dia.loc['Fim de Semana', 'mean'])
            else:
                diferenca = 0.0
        else:
            dia_maior_carga = dia_menor_carga = "N/A"
            diferenca = 0.0
        
        return {
            'dia_maior_carga': dia_maior_carga,
            'dia_menor_carga': dia_menor_carga,
            'diferenca_fds_trabalho': round(diferenca, 2),
            'analise_dias': analise_dia,
            'variacao_diaria': round(analise_dia['mean'].std(), 2) if not analise_dia.empty else 0.0
        }
    
    def _analise_mensal(self, df):
        """Análise de padrões mensais e sazonais"""
        analise_mes = df.groupby('mes')['S'].agg(['mean', 'std']).round(2)
        
        if not analise_mes.empty:
            mes_pico = analise_mes['mean'].idxmax()
            mes_vale = analise_mes['mean'].idxmin()
            amplitude_sazonal = float(analise_mes['mean'].max() - analise_mes['mean'].min())
        else:
            mes_pico = mes_vale = 0
            amplitude_sazonal = 0.0
        
        # Tendência temporal - CORREÇÃO AQUI
        try:
            df_sorted = df.sort_values('datahora')
            df_sorted['data'] = df_sorted['datahora'].dt.date
            tendencia_diaria = df_sorted.groupby('data')['S'].mean()
            
            if len(tendencia_diaria) > 1:
                x = np.arange(len(tendencia_diaria))
                y = tendencia_diaria.values
                coef_tendencia, _ = stats.linregress(x, y)
                
                if coef_tendencia > 0.1:
                    tendencia_geral = 'CRESCENTE'
                elif coef_tendencia < -0.1:
                    tendencia_geral = 'DECRESCENTE'
                else:
                    tendencia_geral = 'ESTÁVEL'
            else:
                coef_tendencia = 0.0
                tendencia_geral = 'INDETERMINADO'
                
        except:
            coef_tendencia = 0.0
            tendencia_geral = 'INDETERMINADO'
        
        return {
            'mes_pico': mes_pico,
            'mes_vale': mes_vale,
            'amplitude_sazonal': round(amplitude_sazonal, 2),
            'tendencia_geral': tendencia_geral,
            'coef_tendencia': round(coef_tendencia, 4),
            'analise_mensal': analise_mes
        }
    
    def _analise_sazonal(self, df):
        """Identificação de padrões sazonais"""
        try:
            # Calcular variação mensal
            carga_mensal = df.groupby(df['datahora'].dt.to_period('M'))['S'].mean()
            
            if len(carga_mensal) > 1:
                variacao_mensal = carga_mensal.pct_change().dropna()
                std_variacao = variacao_mensal.std()
                
                if not np.isnan(std_variacao):
                    if std_variacao > 0.15:
                        padrao_sazonal = "FORTE"
                    elif std_variacao > 0.08:
                        padrao_sazonal = "MODERADO"
                    else:
                        padrao_sazonal = "FRACO"
                else:
                    padrao_sazonal = "INDETERMINADO"
            else:
                padrao_sazonal = "INDETERMINADO (poucos dados)"
                
        except:
            padrao_sazonal = "INDETERMINADO"
        
        return {
            'padrao_sazonal': padrao_sazonal,
            'estabilidade': self._calcular_estabilidade(df)
        }
    
    def _calcular_tendencia_horaria(self, df):
        """Calcula a tendência horária específica - VERSÃO CORRIGIDA"""
        try:
            # Agrupar por hora diretamente do DataFrame
            carga_por_hora = df.groupby('hora')['S'].mean()
            
            # Calcular médias por período
            horas_manha = list(range(6, 12))
            horas_tarde = list(range(12, 18))
            horas_noite = list(range(18, 24))
            horas_madrugada = list(range(0, 6))
            
            carga_manha = carga_por_hora[carga_por_hora.index.isin(horas_manha)].mean()
            carga_tarde = carga_por_hora[carga_por_hora.index.isin(horas_tarde)].mean()
            carga_noite = carga_por_hora[carga_por_hora.index.isin(horas_noite)].mean()
            carga_madrugada = carga_por_hora[carga_por_hora.index.isin(horas_madrugada)].mean()
            
            # Lidar com possíveis NaNs
            carga_manha = carga_manha if not np.isnan(carga_manha) else 0.0
            carga_tarde = carga_tarde if not np.isnan(carga_tarde) else 0.0
            carga_noite = carga_noite if not np.isnan(carga_noite) else 0.0
            carga_madrugada = carga_madrugada if not np.isnan(carga_madrugada) else 0.0
            
        except Exception as e:
            print(f"⚠️ Erro no cálculo de tendência horária: {e}")
            carga_manha = carga_tarde = carga_noite = carga_madrugada = 0.0
        
        return {
            'manha': round(float(carga_manha), 2),
            'tarde': round(float(carga_tarde), 2),
            'noite': round(float(carga_noite), 2),
            'madrugada': round(float(carga_madrugada), 2)
        }
    
    def _calcular_estabilidade(self, df):
        """Calcula métricas de estabilidade"""
        try:
            if len(df) > 0 and df['S'].mean() > 0:
                cv = (df['S'].std() / df['S'].mean()) * 100
                if cv < 15:
                    return "ALTA ESTABILIDADE"
                elif cv < 30:
                    return "ESTABILIDADE MODERADA"
                else:
                    return "ALTA VARIABILIDADE"
            else:
                return "DADOS INSUFICIENTES"
        except:
            return "ERRO NO CÁLCULO"
    
    def _gerar_alertas(self, df):
        """Gera alertas baseados nos padrões identificados"""
        alertas = []
        
        try:
            if len(df) == 0:
                return ["❌ Nenhum dado disponível para análise"]
            
            # Alertas baseados em estatísticas
            carga_media = df['S'].mean()
            carga_maxima = df['S'].max()
            
            if carga_media > 0 and carga_maxima > carga_media * 1.5:
                alertas.append("⚠️ PICOS DE CARGA ELEVADOS: Máximo significativamente acima da média")
            
            cv = (df['S'].std() / df['S'].mean()) * 100
            if cv > 40:
                alertas.append("📊 ALTA VARIABILIDADE: Carga muito instável ao longo do tempo")
            
            # Verificar se há padrões de sobrecarga
            percentil_95 = df['S'].quantile(0.95)
            if percentil_95 > carga_media * 1.3:
                alertas.append("🔴 RISCO DE SOBRECARGA: Valores no percentil 95 significativamente altos")
            
            if len(alertas) == 0:
                alertas.append("✅ Nenhum alerta crítico identificado")
                
        except Exception as e:
            alertas.append(f"⚠️ Erro na geração de alertas: {str(e)}")
        
        return alertas
    
    def _gerar_relatorio_visual(self, resultados, df):
        """Gera visualizações completas do relatório"""
        try:
            fig, axes = plt.subplots(2, 2, figsize=(20, 15))
            fig.suptitle(f'RELATÓRIO DE TENDÊNCIAS - TRAFO {resultados["trafo"]}', fontsize=16, fontweight='bold')
            
            # 1. Comportamento Horário
            analise_hora = df.groupby('hora')['S'].mean()
            if not analise_hora.empty:
                axes[0,0].plot(analise_hora.index, analise_hora.values, marker='o', linewidth=2, markersize=6)
                axes[0,0].axvline(x=resultados['analise_horaria']['hora_pico'], color='red', linestyle='--', alpha=0.7, 
                                label=f'Pico: {resultados["analise_horaria"]["hora_pico"]}h')
                axes[0,0].axvline(x=resultados['analise_horaria']['hora_vale'], color='green', linestyle='--', alpha=0.7, 
                                label=f'Vale: {resultados["analise_horaria"]["hora_vale"]}h')
            axes[0,0].set_xlabel('Hora do Dia')
            axes[0,0].set_ylabel('Carga Média (kVA)')
            axes[0,0].set_title('COMPORTAMENTO HORÁRIO DA CARGA')
            axes[0,0].legend()
            axes[0,0].grid(True, alpha=0.3)
            
            # 2. Carga por Dia da Semana
            dias_ordem = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            analise_dia = df.groupby('dia_semana')['S'].mean()
            # Reindex para garantir ordem correta, preenchendo missing com 0
            analise_dia = analise_dia.reindex(dias_ordem, fill_value=0)
            
            cores = ['blue', 'blue', 'blue', 'blue', 'blue', 'red', 'red']
            bars = axes[0,1].bar(range(len(analise_dia)), analise_dia.values, color=cores, alpha=0.7)
            axes[0,1].set_xlabel('Dia da Semana')
            axes[0,1].set_ylabel('Carga Média (kVA)')
            axes[0,1].set_title('CARGA POR DIA DA SEMANA (Azul=Trabalho, Vermelho=FDS)')
            axes[0,1].set_xticks(range(len(analise_dia)))
            axes[0,1].set_xticklabels(['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sáb', 'Dom'])
            
            # 3. Distribuição por Período do Dia
            periodos = resultados['analise_horaria']['analise_periodo']
            if not periodos.empty:
                axes[1,0].bar(periodos.index, periodos['mean'], yerr=periodos['std'], capsize=5, alpha=0.7)
            axes[1,0].set_xlabel('Período do Dia')
            axes[1,0].set_ylabel('Carga Média (kVA)')
            axes[1,0].set_title('DISTRIBUIÇÃO POR PERÍODO DO DIA')
            axes[1,0].tick_params(axis='x', rotation=45)
            
            # 4. Tendência Temporal
            df_sorted = df.sort_values('datahora')
            carga_diaria = df_sorted.groupby(df_sorted['datahora'].dt.date)['S'].mean()
            if not carga_diaria.empty:
                axes[1,1].plot(carga_diaria.index, carga_diaria.values, linewidth=2)
                if len(carga_diaria) > 1:
                    z = np.polyfit(range(len(carga_diaria)), carga_diaria.values, 1)
                    p = np.poly1d(z)
                    axes[1,1].plot(carga_diaria.index, p(range(len(carga_diaria))), 'r--', alpha=0.8, 
                                 label=f'Tendência: {resultados["analise_mensal"]["tendencia_geral"]}')
            axes[1,1].set_xlabel('Data')
            axes[1,1].set_ylabel('Carga Média Diária (kVA)')
            axes[1,1].set_title('TENDÊNCIA TEMPORAL')
            axes[1,1].legend()
            axes[1,1].tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            os.makedirs('relatorios', exist_ok=True)
            plt.savefig(f'relatorios/relatorio_trafo_{resultados["trafo"]}.png', dpi=300, bbox_inches='tight')
            plt.show()
            
        except Exception as e:
            print(f"⚠️ Erro na geração de gráficos: {str(e)}")
    
    def _exportar_relatorio_textual(self, resultados):
        """Exporta relatório textual detalhado"""
        print(f"\n📈 **RELATÓRIO DETALHADO - TRAFO {resultados['trafo']}**")
        print(f"{'='*60}")
        
        # Resumo Executivo
        print(f"\n🎯 **RESUMO EXECUTIVO:**")
        print(f"   • Horário de Pico: {resultados['analise_horaria']['hora_pico']}h ({resultados['analise_horaria']['carga_pico']} kVA)")
        print(f"   • Horário de Vale: {resultados['analise_horaria']['hora_vale']}h ({resultados['analise_horaria']['carga_vale']} kVA)")
        print(f"   • Amplitude Diária: {resultados['analise_horaria']['amplitude_diaria']} kVA")
        print(f"   • Tendência Geral: {resultados['analise_mensal']['tendencia_geral']}")
        print(f"   • Padrão Sazonal: {resultados['padroes_sazonais']['padrao_sazonal']}")
        
        # Análise Horária Detalhada
        print(f"\n🕒 **PADRÕES HORÁRIOS DETALHADOS:**")
        tendencia = resultados['analise_horaria']['tendencia_horaria']
        print(f"   • Madrugada (0h-6h): {tendencia['madrugada']} kVA")
        print(f"   • Manhã (6h-12h): {tendencia['manha']} kVA")
        print(f"   • Tarde (12h-18h): {tendencia['tarde']} kVA")
        print(f"   • Noite (18h-24h): {tendencia['noite']} kVA")
        
        # Análise Semanal
        print(f"\n📅 **PADRÕES SEMANAIS:**")
        print(f"   • Maior carga: {resultados['analise_diaria']['dia_maior_carga']}")
        print(f"   • Menor carga: {resultados['analise_diaria']['dia_menor_carga']}")
        print(f"   • Diferença FDS vs Trabalho: {resultados['analise_diaria']['diferenca_fds_trabalho']} kVA")
        
        # Alertas e Recomendações
        print(f"\n⚠️ **ALERTAS E RECOMENDAÇÕES:**")
        for alerta in resultados['alertas']:
            print(f"   • {alerta}")
        
        # Recomendações Específicas
        print(f"\n💡 **RECOMENDAÇÕES OPERACIONAIS:**")
        if resultados['analise_horaria']['amplitude_diaria'] > 50:
            print(f"   • ⚡ Considerar readequação de carga devido à alta amplitude diária")
        
        if resultados['analise_mensal']['tendencia_geral'] == 'CRESCENTE':
            print(f"   • 📈 Monitorar crescimento contínuo da demanda")
        
        if "ALTA VARIABILIDADE" in resultados['padroes_sazonais']['estabilidade']:
            print(f"   • 🔄 Implementar controle adaptativo para alta variabilidade")

# ========================================================================
# EXECUÇÃO DO RELATÓRIO - VERSÃO ROBUSTA
# ========================================================================

def gerar_relatorios_completos(data, trafos):
    """Gera relatórios completos para lista de transformadores"""
    print("📊 INICIANDO GERADOR DE RELATÓRIOS DE TENDÊNCIAS")
    print("="*80)
    
    # Verificar se existem dados
    if data is None or len(data) == 0:
        print("❌ ERRO: Nenhum dado fornecido para análise")
        return {}
    
    # Verificar colunas necessárias
    colunas_necessarias = ['id', 'datahora', 'S']
    for col in colunas_necessarias:
        if col not in data.columns:
            print(f"❌ ERRO: Coluna '{col}' não encontrada nos dados")
            return {}
    
    analisador = AnalisadorTendencias(data)
    
    relatorios = {}
    for trafo in trafos:
        try:
            print(f"\n{'='*80}")
            print(f"🔍 ANALISANDO TRAFO {trafo}")
            print(f"{'='*80}")
            
            relatorio = analisador.analisar_tendencias_trafo(trafo)
            if relatorio:
                relatorios[trafo] = relatorio
                print(f"✅ Trafo {trafo} analisado com sucesso!")
            else:
                print(f"❌ Falha na análise do trafo {trafo}")
                
        except Exception as e:
            print(f"❌ Erro crítico ao analisar trafo {trafo}: {str(e)}")
            import traceback
            traceback.print_exc()
    
    print(f"\n✅ Relatórios gerados para {len(relatorios)} transformadores")
    print("📁 Arquivos salvos na pasta 'relatorios/'")
    
    return relatorios

# Execute o teste se rodar diretamente
if __name__ == "__main__":
    
    # Depois com seus dados reais (descomente se quiser)
    # print("\n2. 📊 Análise com dados reais...")
    relatorios_reais = gerar_relatorios_completos(df_hourly, ['T1', 'T2', 'T3'])

In [ ]:
relatorios_reais = gerar_relatorios_completos(df_hourly, ['T1', 'T2', 'T3'])

# 3. Acesse os insights
for trafo, relatorio in relatorios_reais.items():
    print(f"\n💡 INSIGHT {trafo}:")
    print(f"   Pico: {relatorio['analise_horaria']['hora_pico']}h")
    print(f"   Vale: {relatorio['analise_horaria']['hora_vale']}h") 
    print(f"   Tendência: {relatorio['analise_mensal']['tendencia_geral']}")